In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# Maximum Likelihood on Lie Groups
Exemplified on SO(3)

In [ ]:
# SO(3)
from src.groups.SON import *
G = SON(3)
print(G)

# SO(3) acts on S^2
from src.manifolds.S2 import *
M = S2()
print(M)

from src.plotting import *

In [ ]:
from src.group import invariant_metric
invariant_metric.initialize(G)

from src.group import energy
energy.initialize(G)

In [ ]:
q = np.array([1e-6,0,0])
g = G.psif(q)
v = np.array([0,1,1])

In [ ]:
# sample data
from src.stochastics import Brownian_inv
Brownian_inv.initialize(G)

G.sigma.set_value(np.diag((.3,.1,.7))) # set metric
K = 64
obss = np.zeros((K,)+g.shape)
# srng.seed(422)
for i in range(K):
    (ts,gs) = G.Brownian_invf(g,dWsf(G.dim.eval()))
    obss[i] = gs[-1]

# plot samples
newfig()
for i in range(K):
    G.plotg(obss[i])
plt.show()

# on S2
x = np.array([0,0,1])
obss_M = np.zeros((K,M.emb_dim.eval(),))
newfig()
M.plot()
for i in range(K):
    obss_M[i] = M.actf(obss[i],x)
    M.plotx(obss_M[i])
plt.show()

In [ ]:
# plot estimated density        
newfig()
# M.plot(alpha=.4)
plot_density_estimate(M,obss_M)
plt.show()

In [ ]:
# fiber Brownian motion
from src.group.quotient import *
sde_Brownian_inv_fiber = get_sde_fiber(G.sde_Brownian_inv,lambda g: M.act(g,x),G,M)
Brownian_inv_fiber = lambda g,dWt: integrate_sde(sde_Brownian_inv_fiber,
                                                  integrator_stratonovich,
                                                  g,dWt)
g0 = G.element()
Brownian_inv_fiberf = theano.function([g0,dWt], Brownian_inv_fiber(g0,dWt))

(ts,gs) = Brownian_inv_fiberf(g,dWsf(G.dim.eval()))
newfig()
G.plotg(gs)
plt.show()

# on S2
newfig()
M.plot()
M.plotx(M.actsf(gs.transpose((1,2,0)),x).T)
plt.show()

In [ ]:
# find g in fiber above x closests to g0
xx = M.actf(G.psif(v),x)
print(xx)

(lift,mu_lift) = lift_to_fiber(xx,x,G,M)
print(M.actf(lift,x))

(ts,gs) = G.exptf(G.VtoLAf(mu_lift))
newfig()
G.plotg(gs)
G.plotg(lift,color='k')
plt.show()

# on S2
newfig()
M.plot()
M.plotx(M.actsf(gs.transpose((1,2,0)),x).T)
M.plotx(xx,color='k')
plt.show()

In [ ]:
# Delyon/Hu guided process
from src.stochastics.guided_process import *

# parameters
thetas = (G.sigma,)
thetas_true = [theta.eval() for theta in thetas]

# guide function
# phi = lambda g,v: -(g-v)
phi = lambda g,v: T.tensordot(G.inv(G.sigma),G.LAtoV(G.log(G.invtrns(G.inv(g),v))),(1,0))

(dlog_likelihood,dlog_likelihoodf,Brownian_inv_guided,Brownian_inv_guidedf) = get_guided_likelihood(G,G.sde_Brownian_inv,phi,lambda g: G.sigma, g0, thetas, A=G.gG, integration='stratonovich')

w = G.psif(v)
(ts,gs,log_likelihood,log_varphi) = Brownian_inv_guidedf(g,w,dWsf(G.dim.eval()))
print("log likelihood: ", log_likelihood[-1], ", log varphi: ", log_varphi[-1])

newfig()
G.plotg(gs)
G.plotg(w,color='k')
plt.show()

# on S2
newfig()
M.plot()
M.plotx(M.actsf(gs.transpose((1,2,0)),x).T)
M.plotx(M.actf(w,x),color='k')
plt.show()

In [ ]:
options = {}
options['samples_per_obs'] = 1
options['epochs'] = 200
options['learning_rate'] = 2.5*1.e-3
options['varphi_update_rate'] = 1.
options['initial'] = (np.diag((.08,.1,.2)),)#.1*np.eye(G_dim.eval())

In [ ]:
# Transition density
g0 = G.element()
v = G.element()
log_p_Tf = theano.function([g0,v],log_p_T(g0,v,dWs(G.dim),Brownian_inv_guided,phi,options,sde=G.sde_Brownian_inv,sigma=G.sigma))
dlog_p_Tf = theano.function([g0,v],dlog_p_T(thetas,g0,v,dWs(G.dim),Brownian_inv_guided,phi,options,sde=G.sde_Brownian_inv,sigma=G.sigma))
p_Tf = theano.function([g0,v],T.exp(log_p_T(g0,v,dWs(G.dim),Brownian_inv_guided,phi,options,sde=G.sde_Brownian_inv,sigma=G.sigma)))

In [ ]:
# sample bridges
def lbridge_sampling(thetas,*args,**kwargs):
    G.sigma.set_value(thetas[0])
    return partial(bridge_sampling,g,dlog_likelihoodf,lambda: dWsf(G.dim.eval()),options)(*args,**kwargs)

log_phis = np.zeros((K,))
try:
    mpu.openPool()
    sol = mpu.pool.imap(partial(lbridge_sampling,options['initial']),mpu.inputArgs(obss,np.random.randint(1000,size=K)))
    res = list(sol)
    bridges = mpu.getRes(res,0)
    log_varphis = mpu.getRes(res,1)
    log_likelihoods = mpu.getRes(res,2)
except:
    mpu.closePool()
    raise
else:
    mpu.closePool()

# Plot on S2
newfig()
M.plot()
colormap = plt.get_cmap('winter')
colors=[colormap(k) for k in np.linspace(0, 1, K)]
for i in range(bridges.shape[0]):
    M.plotx(M.actf(obss[i],x),color=colors[i])
    for j in range(bridges.shape[1]):
        gs = bridges[i,j]
        M.plotx(M.actsf(gs.transpose((1,2,0)),x).T,linewidth=.2,color=colors[i])
plt.show()

In [ ]:
%%time
from src.statistics.mle import *

def llog_p_T(thetas,pars):
    (v,seed) = pars
    if seed:
        srng.seed(seed)    
    G.sigma.set_value(thetas[0])
    return dlog_p_Tf(g,v)

def update_thetas(thetas, dthetas):
    sigma = thetas[0]
    sigma += options['learning_rate']*dthetas[0]
    
    return (sigma,)

# run MLE
(thetas, log_likelihood, log_likelihoods, thetass) = iterative_mle(obss,llog_p_T,update_thetas,options)

# plot
plt.plot(range(options['epochs']),log_likelihoods)
plt.show()
plt.plot(range(options['epochs']),thetass[0].reshape((thetass[0].shape[0],-1)))
plt.hlines(thetas_true[0].flatten(),plt.xlim()[0],plt.xlim()[1],color='r')
plt.show()
None